In [113]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv")
df.head()

,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,2.72,14,4.53,0.6,2.718,87.93,0.6,52.758,92.46,5.0,...,0.4,0,0.2,0.0,4.0,108.362,0.4,0.5,21.6724,1
1,3.41,16,12.68,0.6,7.608,41.00,0.6,24.600,53.68,5.5,...,0.4,0,0.2,0.0,4.4,41.936,0.4,0.5,8.3872,1
2,2.72,11,3.11,0.6,1.866,113.97,0.6,68.382,117.08,5.5,...,0.4,0,0.2,0.0,4.4,88.832,0.4,0.5,17.7664,1
3,3.41,8,1.12,0.4,0.448,0.00,0.2,0.000,1.12,5.0,...,0.4,0,0.2,0.0,2.2,1.848,0.4,0.5,0.3696,0
4,55.57,9,1.06,0.4,0.424,0.00,0.2,0.000,1.06,5.0,...,0.4,0,0.2,0.0,2.2,1.824,0.4,0.5,0.3648,0


In [114]:
#1 데이터의 Risk 값에 따른 score_a와 score_b의 평균값을 구하여라
df.groupby("Risk").mean()[['Score_A','Score_B']]

,Score_A,Score_B
Risk,,
0,0.262234,0.225532
1,0.490164,0.444262


In [115]:
import pandas as pd
# 데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")


In [116]:
x_train = x_train.merge(y_train, on = 'CustomerId')

In [117]:
((x_train['CustomerId'] == y_train['CustomerId']).sum()) == len(x_train)

True

In [118]:
x_train.drop(columns = ["Surname",'CustomerId'],inplace= True)

In [119]:
x_train.nunique()

CreditScore         459
Geography             3
Gender                4
Age                  69
Tenure               11
Balance            4162
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    6499
Exited                2
dtype: int64

In [120]:
x_train['Gender'] = x_train['Gender'].apply(lambda x : x.lower())
x_train['Gender'] = x_train['Gender'].str.replace(" ","")

pd.get_dummies(x_train["Gender"])

,female,male
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
...,...,...
6494,0,1
6495,0,1
6496,1,0
6497,1,0


In [121]:
x_train.groupby(['Geography','Exited']).count()['Gender']

Geography  Exited
France     0         2711
           1          516
Germany    0         1112
           1          538
Spain      0         1352
           1          270
Name: Gender, dtype: int64

In [122]:
x_train = x_train.join(pd.get_dummies(x_train['Geography']))
x_train = x_train.join(pd.get_dummies(x_train['Gender']))
x_train.drop(columns = ["Geography","Gender"],inplace=True)

In [123]:
x_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain,female,male
0,791,35,7,52436.20,1,1,0,161051.75,0,0,1,0,1,0
1,705,42,8,166685.92,2,1,1,55313.51,0,0,1,0,0,1
2,543,31,4,138317.94,1,0,0,61843.73,0,1,0,0,1,0
3,709,32,2,0.00,2,0,0,109681.29,0,1,0,0,1,0
4,714,36,1,101609.01,2,1,1,447.73,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6494,696,24,9,0.00,1,0,0,10883.52,0,0,0,1,0,1
6495,513,34,7,60515.13,1,0,0,124571.09,0,0,1,0,0,1
6496,663,22,9,0.00,1,1,0,29135.89,1,0,0,1,1,0
6497,635,48,2,0.00,2,1,1,136551.25,0,0,0,1,1,0


In [124]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score,f1_score

model = RandomForestClassifier()
kfold = KFold()
#x_train.drop(inplace = True, columns = "Exited")
acc = []
f1 = []
for train_index, test_index in kfold.split(x_train):
    model.fit(x_train.iloc[train_index],y_train.iloc[train_index]['Exited'])
    predict_value = model.predict(x_train.iloc[test_index])
    acc.append(accuracy_score(y_train.iloc[test_index]['Exited'],predict_value))
    f1.append(f1_score(y_train.iloc[test_index]['Exited'],predict_value))
    

In [125]:
import numpy as np
np.average(acc), np.average(f1)

(1.0, 1.0)

In [126]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

for train_index, test_index in kfold.split(x_train):
    model.fit(x_train.iloc[train_index],y_train.iloc[train_index]['Exited'])
    predict_value = model.predict(x_train.iloc[test_index])
    acc.append(accuracy_score(y_train.iloc[test_index]['Exited'],predict_value))
    f1.append(f1_score(y_train.iloc[test_index]['Exited'],predict_value))
    
np.mean(acc),np.mean(f1)

(0.8957534197903714, 0.5599095343256154)

In [127]:
y_train['Exited'].value_counts()

0    5175
1    1324
Name: Exited, dtype: int64

In [128]:
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
x_train.drop(columns = ["Surname",'CustomerId'],inplace= True)

x_train['Gender'] = x_train['Gender'].apply(lambda x : x.lower())
x_train['Gender'] = x_train['Gender'].str.replace(" ","")


In [129]:
from sklearn.preprocessing import LabelEncoder
gender_lab = LabelEncoder()
geo_lab = LabelEncoder()

gender_lab.fit(x_train['Gender'])
x_train['Gender'] = gender_lab.transform(x_train['Gender'])

geo_lab.fit(x_train['Geography'])
x_train['Geography'] = geo_lab.transform(x_train['Geography'])

In [130]:
x_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,791,1,0,35,7,52436.20,1,1,0,161051.75
1,705,1,1,42,8,166685.92,2,1,1,55313.51
2,543,0,0,31,4,138317.94,1,0,0,61843.73
3,709,0,0,32,2,0.00,2,0,0,109681.29
4,714,1,0,36,1,101609.01,2,1,1,447.73
...,...,...,...,...,...,...,...,...,...,...
6494,696,2,1,24,9,0.00,1,0,0,10883.52
6495,513,1,1,34,7,60515.13,1,0,0,124571.09
6496,663,2,0,22,9,0.00,1,1,0,29135.89
6497,635,2,0,48,2,0.00,2,1,1,136551.25


In [131]:
y_train

,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0
...,...,...
6494,15702806,0
6495,15674179,0
6496,15790204,1
6497,15690772,0


In [132]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score,f1_score

model = RandomForestClassifier()
kfold = KFold()
#x_train.drop(inplace = True, columns = "Exited")
acc = []
f1 = []
for train_index, test_index in kfold.split(x_train):
    model.fit(x_train.iloc[train_index],y_train.iloc[train_index]['Exited'])
    predict_value = model.predict(x_train.iloc[test_index])
    acc.append(accuracy_score(y_train.iloc[test_index]['Exited'],predict_value))
    f1.append(f1_score(y_train.iloc[test_index]['Exited'],predict_value))
    

In [133]:
np.mean(acc), np.mean(f1)

(0.862132291111506, 0.572417660011743)

In [134]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

for train_index, test_index in kfold.split(x_train):
    model.fit(x_train.iloc[train_index],y_train.iloc[train_index]['Exited'])
    predict_value = model.predict(x_train.iloc[test_index])
    acc.append(accuracy_score(y_train.iloc[test_index]['Exited'],predict_value))
    f1.append(f1_score(y_train.iloc[test_index]['Exited'],predict_value))
    
np.mean(acc),np.mean(f1)

(0.8272042991650382, 0.3445977002613926)

In [ ]:
from sklearn.pipeline import Pipeline
